# Spark SQL

## 1. Introduction to DataFrames and Datasets

### Difference between RDDs, DataFrames and DataSets.

**RDDs (2011)**:
- Distribute collection of Java Virtual Machine (JVM) objects
- Functional Operators (map, reduce, filter, etc)

**DataFrame (2013)**:
- Distributed collection of Row objects
- Expression based operations and User Defined Functions (UDFs)
- Logical plans and optimizer
- Fast/efficient internal representations

**DataSet (2015)**:
- Internally Rows, externally JVM objects
- Almost the "Best of both worlds": type safe + fast
- Slower than DataFrame, not as good for interactive analysis, especially Python

## 2. DataFrames

<img src="images/dataframe_concept.png" title="DataFrame Concept" width="700px"/>

### Basic concepts
- Distributed collection of Row objects. These objects contain the schema within the data.
- Data is organized into columns like a relational database.
- The main features of Dataframes:
- **Catalyst**: powers the Dataframe and SQL APIs.
    1. Analyzing a logical plan to resolve references
    2. Logical plan optimization
    3. Physical planning
    4. Code generation to compile parts of the query to Java bytecode.
- **Tungsten**: provides a physical execution backend which explicitly manages memory and dynamically generates bytecode for expression evaluation.


### Creating DataFrames

DataFrames can be created from many different sources such as existing RDDs, structured and unstructured data files (CSV, JSON, Parquet), databases using JDBC, etc.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import Row, IntegerType

# Initialize Spark session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Creating DataFrame from an existing RDD
rdd = spark.sparkContext.parallelize([0, 1, 2, 3, 4, 5])
df = rdd.map(lambda x: Row(x)).toDF()
print("DataFrame from RDD.toDF() function")
df.show()
schema = IntegerType()
df = spark.createDataFrame(rdd, schema)
print("DataFrame from RDD with saprk.createDataFrame()")
df.show()

# Creating a DataFrame from a Python collection
cars = [
    {"brand": "Toyota", "name": "Camry", "price": 24000},
    {"brand": "Honda", "name": "Civic", "price": 22000},
    {"brand": "Ford", "name": "Mustang", "price": 27000},
    {"brand": "Tesla", "name": "Model 3", "price": 35000},
    {"brand": "Chevrolet", "name": "Malibu", "price": 23000}
]
df = spark.createDataFrame(cars)
print("DataFrame from a Python collection")
df.show()

# Creating a DataFrame from a JSON file
df = spark.read.json('datasets/students.json')
print("DataFrame from a JSON file")
df.show()

## 3. Running SQL Queries using Spark SQL

In Spark you can work with DataFrames using the built-in functions or the Spark SQL API that provides a syntax similar to standard SQL but includes many extra ones that can be found [here](https://spark.apache.org/docs/latest/api/sql/index.html) (I recommend you to add this page to your bookmarks!). 

`NOTE`: Depending on the Spark version you are using this functions may change, you can find all the available Spark Docs for the different versions [here](https://spark.apache.org/docs/). To check you Spark version print your Spark Session.

In [ ]:
spark

In [ ]:
# First, lets check the schema of the df we will work with
df.printSchema()

### Reading the top 10 lines

#### DataFrame mode

#### SQL mode

### Understanding the Pyshical and Logical plans

As previously stated, the **Catalyst** is in charge of creating an optimizing the plan that will be converted into a DAG and sent to the executors. The following diagram shows the steps that the Catalyst performs to produce an execution plan

<img src="images/catalyst_steps.webp" title="Catalyst Steps Diagram" width="700px"/>

In order to see the plan generated by the **Catalyst** you can call the function `explain()` which, by default, will display the Physical Plan.

#### Getting various plans
Before Apache Spark 3.0, there was only two modes available to format explain output.

- `explain(extended=False)` which displayed only the physical plan
- `explain(extended=True)` which displayed all the plans (logical and physical)

Starting from Apache Spark 3.0, you have a new parameter “mode” that produce expected format for the plan:

- `explain(mode="simple")` which will display the physical plan
- `explain(mode="extended")` which will display physical and logical plans (like "extended" option)
- `explain(mode="codegen")` which will display the java code planned to be executed
- `explain(mode="cost")` which will display the optimized logical plan and related statistics (if they exist)
- `explain(mode="formatted")` which will display a splitted output composed by a nice physical plan outline, and a section with each node details

### Top 5 students with higher grades

#### DataFrame mode

#### SQL mode

### Max and Mean grade

#### DataFrame mode

#### SQL mode

### Top 5 students with highest mean grade

#### DataFrame mode

#### SQL mode

### Average grade per student age

#### DataFrame mode

#### SQL mode

### Add column `excelent` to indicate that a student has a grade over 9.5

#### DataFrame mode

#### SQL mode

## 4. Spark SQL Joins

Spark DataFrame supports all basic SQL Join Types like `INNER`, `LEFT OUTER`, `RIGHT OUTER`, `LEFT ANTI`, `LEFT SEMI`, `CROSS`, `FULL OUTER`, etc. Spark SQL Joins are wide transformations that result in data shuffling over the network hence they have huge performance issues when not designed with care.

On the other hand Spark SQL Joins comes with more optimization by default (thanks to DataFrames & Dataset) however still there would be some performance issues to consider while using them.

### Most common types of Joins

<img src="images/joins.jpg" title="Types of Joins" width="700px"/>

### How to perform a Join in Spark (cheat sheet)

1. Referencing columns with different names in each df:

```python
dfA.join(dfB, dfA["idA"] == dfB["idB"], "type-of-join")

```

2. Key column has the same alias

```python
dfA.join(dfB, on="id", how="type-of-join")

```

3. More than one key column

```python
dfA.join(dfB, ["id", "code"], "type-of-join")

dfA.join(dfB, (dfA["id"] == dfB["id"]) & (dfA["code"] == dfB["code"]), "type-of-join")

```

4. Maintaining both columns (be careful and be sure of referencing the origin of the column when using it after this)

```python
dfC = dfA.join(dfB, dfA["id"] == dfB["id"], "type-of-join")

dfC.select(dfA["id"]).show()
```

### Read the example data

In [ ]:
df_emp = spark.read.json("datasets/employees.json")
df_emp.show()

In [ ]:
df_dept = spark.read.json("datasets/departments.json")
df_dept.show()

### Inner Join

- The employee that belongs to inexistent department with id 50 is dropped
- The Sales department that has no employees asociated is dropped

### Full Outer Join

Outer a.k.a full, fullouter join returns all rows from both Spark DataFrame, where join expression doesn’t match it returns null on respective record columns.

- There is no department with id 50 hence the values for the department are null
- There are no employees in the Sales department hence the employee values are null

### Left Join

Left a.k.a Left Outer join returns all rows from the left DataFrame/Dataset regardless of match found on the right dataset when join expression doesn’t match, it assigns null for that record and drops records from right where match not found.

- There is no department with id 50 hence the values for the department are null
- There are no employees in the Sales department hence no row is shown

### Right Join

Right a.k.a Right Outer join is opposite to left join, here it returns all rows from the right DataFrame/Dataset regardless of match found on the left dataset, when join expression doesn’t match, it assigns null for that record and drops records from left where match not found.

- There is no department with id 50 hence no row is shown
- There are no employees in the Sales department hence the values of the employee columns are null

### CrossJoin

This join combines each row of the first table with each row of the second table. For example, we have `m` rows in one table and `n` rows in another, this gives us `m * n` rows in the resulting table. 

`Note`: A table of 1000 customers combined with a table of 1000 products would produce 1,000,000 records! Try to avoid this with large tables in production.

## 5. User Defined Functions (UDFs)
User Defined Functions (UDFs) in Spark allow users to define their own transformations using Python or other programming languages, and then apply those functions on a Spark DataFrame. This can be very powerful when you need to make specific transformations to your data that aren't easily achieved using Spark's built-in functions.

When a UDF is defined, under the hood, Spark serializes the function using Py4J, transfers it over to the executor nodes, and deserializes it. This allows the UDF to be executed on rows of the DataFrame in parallel. However, it's worth noting that because UDFs involve serialization and data transfer between Python and JVM, they can be considerably slower than using native Spark functions.

Here's how you can define and use a UDF in Spark with Python (PySpark):

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def add_suffix(name):
    return name + "_UDF"

suffix_udf = udf(add_suffix, StringType())

#### DataFrame Mode

#### SQL Mode